In [ ]:
import numpy as np
import pandas as pd
import glob
import os
from scipy.stats import pearsonr
import anndata as ad
from scipy.sparse import csr_matrix
from scipy.stats import zscore
import time
from scipy.sparse import issparse

In [8]:
scrna_path = "/mnt/c/Users/donna/Downloads/Thesis/data/scrnatest"

In [9]:
reference_df = pd.read_excel("/mnt/c/Users/donna/Downloads/Thesis/rankjes/paper_data.xlsx", usecols=["cell_type", "tissue", "rank", "GC_code", "depth"])
reference_df.dropna(subset=["cell_type", "tissue"], inplace=True)
reference_df["normalized_cell_type"] = [name.rsplit("_", 1)[0] for name in reference_df["cell_type"]]
reference_df["normalized_tissue"] = reference_df["tissue"].str.lower().str.replace(r'[-\s]+', '_', regex=True)

In [10]:
h5ad_files = [f for f in os.listdir(scrna_path) if f.endswith(".h5ad")]

In [11]:
def strip_suffix(cell_type):
        for suffix in suffixes_to_remove:
            if cell_type.endswith(suffix):
                return cell_type[: -len(suffix)]
        return cell_type

In [6]:
filtered_rows = []


for file_name in h5ad_files:

    full_path = os.path.join(scrna_path, file_name)
    adata = ad.read_h5ad(full_path)

    adata.obs['cell_type'] = [name.replace(", ", "_").replace(" ", "_").replace("-", "_") for name in adata.obs['cell_type']]

    tissue_name = os.path.basename(file_name).replace(".h5ad", "")
    
    suffixes_to_remove = ["_Bone", "_Small", "_Large", "_Salivary", "_Lymph"]
    filtered_reference = reference_df[reference_df["normalized_tissue"] == tissue_name].copy()
    filtered_reference["normalized_cell_type"] = filtered_reference["normalized_cell_type"].apply(strip_suffix)

    valid_cell_types = set(filtered_reference["normalized_cell_type"].unique())
    current_cell_types = set(adata.obs['cell_type'].unique())
    matching_types = current_cell_types & valid_cell_types

    filtered_reference = filtered_reference[filtered_reference["normalized_cell_type"].isin(matching_types)]

    filtered_rows.append(filtered_reference)


matched_reference_df = pd.concat(filtered_rows, ignore_index=True)

# Filter for only 10x coverage 
filtered_depth_df = matched_reference_df[matched_reference_df["depth"] == "<0.3X"].copy()



In [12]:
original_columns = filtered_depth_df.columns.tolist()
sorted_df = filtered_depth_df.sort_values(by=["GC_code", "rank"])

In [13]:
def assign_compressed_ranks(group):
    group = group.copy()
    group["rank"] = range(1, len(group) + 1)  # assign new ranks 1..n
    return group

In [ ]:
recomputed_df = (
    sorted_df.groupby("GC_code", group_keys=False)
    .apply(assign_compressed_ranks)
)
recomputed_df = recomputed_df[original_columns]


/tmp/ipykernel_18390/1499007839.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(assign_compressed_ranks)


: 

In [ ]:
recomputed_df.to_csv("rankjes/matched_reference_recomputed_ranks.csv", index=False)

In [15]:
df = pd.read_csv("/mnt/c/Users/donna/Downloads/Thesis/rankjes/paper_ranks/03x_matched_reference_ranks.csv")

In [16]:
two_word_tissues = {
    'Small_Intestine',
    'Large_Intestine',
    'Salivary_Gland',
    'Bone_Marrow',
    'Lymph_Node'
}

In [17]:
processed_rows = []

for _, row in df.iterrows():
    cell_type = row['cell_type']
    rank = row['rank'] if 'rank' in row else None  # Skip or adjust if rank is not present

    parts = cell_type.split('_')

    if len(parts) >= 2:
        last_two = '_'.join(parts[-2:])
        last_one = parts[-1]

        if last_two in two_word_tissues:
            tissue = last_two
            original_cell_type = '_'.join(parts[:-2])
        else:
            tissue = last_one
            original_cell_type = '_'.join(parts[:-1])
    else:
        tissue = 'unknown'
        original_cell_type = cell_type

    # Store modified row
    processed_row = row.to_dict()
    processed_row['tissue'] = tissue
    processed_row['original_cell_type'] = original_cell_type
    processed_rows.append(processed_row)

# Create a new DataFrame
processed_df = pd.DataFrame(processed_rows)

# Save to new CSV
output_path = "/mnt/c/Users/donna/Downloads/Thesis/rankjes/paper_ranks/03x_matched_reference_ranks.csv"  # replace with your desired output path
processed_df.to_csv(output_path, index=False)